# Use Machine Learning in a Trading Strategy

## Loading and Preparing the Data

In [6]:
# Import required libraries
import pandas as pd
from pathlib import Path

In [7]:
# Import the OHLCV dataset into a Pandas Dataframe
trading_df = pd.read_csv(
    Path("../Resources/ohlcv.csv"), 
    index_col="date", 
    infer_datetime_format=True, 
    parse_dates=True
)

# Display sample data
trading_df.head()

open   high    low   close  volume
date                                                    
2018-10-19 09:30:00  16.90  17.18  16.90  17.095   11522
2018-10-19 09:45:00  17.11  17.44  17.11  17.400   70593
2018-10-19 10:00:00  17.40  17.40  17.25  17.280   38885
2018-10-19 10:15:00  17.27  17.27  17.18  17.200   37046
2018-10-19 10:30:00  17.21  17.37  17.19  17.200   46874

In [8]:
# Calculate the daily returns using the closing prices and the pct_change function
trading_df["actual_returns"] = trading_df["close"].pct_change()

# Display sample data
trading_df.head()

open   high    low   close  volume  actual_returns
date                                                                    
2018-10-19 09:30:00  16.90  17.18  16.90  17.095   11522             NaN
2018-10-19 09:45:00  17.11  17.44  17.11  17.400   70593        0.017841
2018-10-19 10:00:00  17.40  17.40  17.25  17.280   38885       -0.006897
2018-10-19 10:15:00  17.27  17.27  17.18  17.200   37046       -0.004630
2018-10-19 10:30:00  17.21  17.37  17.19  17.200   46874        0.000000

In [9]:
# Drop all NaN values from the DataFrame
trading_df = trading_df.dropna()

# Review the DataFrame
display(trading_df.head())
display(trading_df.tail())

open   high    low  close  volume  actual_returns
date                                                                   
2018-10-19 09:45:00  17.11  17.44  17.11  17.40   70593        0.017841
2018-10-19 10:00:00  17.40  17.40  17.25  17.28   38885       -0.006897
2018-10-19 10:15:00  17.27  17.27  17.18  17.20   37046       -0.004630
2018-10-19 10:30:00  17.21  17.37  17.19  17.20   46874        0.000000
2018-10-19 10:45:00  17.20  17.20  17.10  17.12   11266       -0.004651

open  high    low  close  volume  actual_returns
date                                                                  
2020-09-04 14:45:00  6.225  6.26  6.220  6.250   55512        0.003210
2020-09-04 15:00:00  6.255  6.27  6.245  6.250   65810        0.000000
2020-09-04 15:15:00  6.250  6.29  6.250  6.275  202630        0.004000
2020-09-04 15:30:00  6.270  6.28  6.250  6.255  130140       -0.003187
2020-09-04 15:45:00  6.250  6.28  6.250  6.250  190278       -0.000799

## Generating the Features and Target Sets

### Creating the Features Set

In [10]:
# Define a window size of 4
short_window = 4

# Create an SMA that uses short_window, and assign it to a new column named “sma_fast”
trading_df["sma_fast"] = trading_df["close"].rolling(window=short_window).mean()

In [11]:
# Define a window size of 20
long_window = 20

# Create an SMA that uses long_window, and assign it to a new columns named “sma_slow”
trading_df["sma_slow"] = trading_df["close"].rolling(window=long_window).mean()

In [12]:
# Drop the NaNs using dropna()
trading_df = trading_df.dropna()

In [13]:
# Assign a copy of the `sma_fast` and `sma_slow` columns to a new DataFrame called `X`
X = trading_df[["sma_fast", "sma_slow"]].shift().dropna().copy()

# Display sample data
display(X.head())
display(X.tail())

sma_fast  sma_slow
date                                   
2018-10-19 14:45:00  16.97750  17.02875
2018-10-19 15:00:00  16.99250  17.01125
2018-10-19 15:15:00  17.01375  16.99825
2018-10-19 15:30:00  17.03375  16.99100
2018-10-19 15:45:00  17.04875  16.98450

sma_fast  sma_slow
date                                   
2020-09-04 14:45:00   6.22625   6.17150
2020-09-04 15:00:00   6.22875   6.17300
2020-09-04 15:15:00   6.23875   6.17700
2020-09-04 15:30:00   6.25125   6.18375
2020-09-04 15:45:00   6.25750   6.19350

### Creating the Target Set

In [14]:
# Create a new column in the `trading_df` called "signal" setting its value to zero.
trading_df["signal"] = 0.0

In [15]:
# Create the signal to buy
trading_df.loc[(trading_df["actual_returns"] >= 0), "signal"] = 1

In [16]:
# Create the signal to sell
trading_df.loc[(trading_df["actual_returns"] < 0), "signal"] = -1

In [17]:
# Copy the new "signal" column to a new Series called `y`.
y = trading_df["signal"].copy()

## Split the Data into Training and Testing Sets

In [18]:
# Import required libraries
from pandas.tseries.offsets import DateOffset

In [19]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2018-10-19 14:45:00


In [20]:
# Select the ending period for the training data with an offset of 18 months
training_end = X.index.min() + DateOffset(months=18)

# Display the training end date
print(training_end)

2020-04-19 14:45:00


In [21]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

In [22]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

## Standardize the Data

In [23]:
# Import required libraries
from sklearn.preprocessing import StandardScaler

In [24]:
# Create a StandardScaler instance
scaler = StandardScaler()
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Using Machine Learning Into a Trading Strategy

In [20]:
# Import the SVM model


In [5]:
# Create the classifier model

 
# Fit the model to the data using X_train_scaled and y_train


# Use the trained model to predict the trading signals for the training data


# Display the sample predictions


In [4]:
# Evaluate the model using a classification report


# Display the report


### Backtesting a Machine Learning Trading Algorithm

In [23]:
# Use the trained model to predict the trading signals for the testing data.


In [3]:
# Evaluate the model's ability to predict the trading signal for the testing data


# Display the report


### Visually Compare the Actual and Predicted Returns

In [2]:
# Create a predictions DataFrame


# Review the DataFrame


In [1]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
